In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from attention_graph_util import *
import seaborn as sns
import itertools 
import matplotlib as mpl
import networkx as nx
import os
from util import constants

from absl import app
from absl import flags
import pandas as pd

from util.models import MODELS
from util.tasks import TASKS
#from dnotebook_utils import *
from attention_graph_util import *
%matplotlib inline
from util.config_util import get_task_params
from notebooks.notebook_utils import *
from util import inflect

from tqdm import tqdm
from scipy.stats import spearmanr
import math


rc={'font.size': 10, 'axes.labelsize': 10, 'legend.fontsize': 10.0, 
    'axes.titlesize': 32, 'xtick.labelsize': 20, 'ytick.labelsize': 16}
plt.rcParams.update(**rc)
mpl.rcParams['axes.linewidth'] = .5 #set the value globally

import torch
from transformers import *
from transformers import BertConfig, BertForMaskedLM, BertTokenizer

[nltk_data] Downloading package punkt to /home/dehghani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
!pip install --upgrade transformers
!pip install networkx
!pip install --upgrade matplotlib
!pip install --upgrade seaborn


!pip install torch torchvision

Requirement already up-to-date: transformers in /home/dehghani/anaconda3/envs/indist/lib/python3.7/site-packages (2.8.0)
Requirement already up-to-date: matplotlib in /home/dehghani/anaconda3/envs/indist/lib/python3.7/site-packages (3.2.1)
Requirement already up-to-date: seaborn in /home/dehghani/anaconda3/envs/indist/lib/python3.7/site-packages (0.10.0)


In [3]:
task_name = 'word_sv_agreement_lm'
task_params = get_task_params(batch_size=1)
task = TASKS[task_name](task_params, data_dir='../InDist/data')
cl_token = task.sentence_encoder().encode(constants.bos)
task_tokenizer = task.sentence_encoder()._tokenizer

INFO:absl:Overwrite dataset info from restored data version.


Vocab len:  10032


INFO:absl:Constructing tf.data.Dataset for split validation, from ../InDist/data/word_sv_agreement/0.1.0
INFO:absl:Constructing tf.data.Dataset for split test, from ../InDist/data/word_sv_agreement/0.1.0
INFO:absl:Constructing tf.data.Dataset for split train, from ../InDist/data/word_sv_agreement/0.1.0


In [4]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForMaskedLM.from_pretrained('distilbert-base-uncased',
                                        output_hidden_states=True,
                                        output_attentions=True)

In [5]:
def offset_convertor(encoded_input_task, task_offset, task_encoder, tokenizer):
    string_part1 = task_encoder.decode(encoded_input_task[:task_offset])
    tokens_part1 = tokenizer.tokenize(string_part1)
    
    return len(tokens_part1)


In [6]:
for x,y in task.test_dataset:
    sentence = task.sentence_encoder().decode(x[0][1:])
    print(sentence)
    break

tokens = ['cls']+tokenizer.tokenize(sentence)+['sep']
print(len(tokens), tokens)
tf_input_ids = tokenizer.encode(sentence)
input_ids = torch.tensor([tf_input_ids])
logits, all_hidden_states, all_attentions = model(input_ids)
print(logits.shape)
_attentions = [att.detach().numpy() for att in all_attentions]
attentions_mat = np.asarray(_attentions)[:,0]
print(attentions_mat.shape)

embeded_inputs = torch.autograd.Variable(model.distilbert.embeddings(input_ids), requires_grad=True)
logits, all_hidden_states, all_attentions = model(inputs_embeds=embeded_inputs)
print(embeded_inputs.shape)


lsum = logits.sum()
print(lsum)

lsum.backward()
embeded_inputs.require_grad = True
print(embeded_inputs.grad.shape)

many NNS of woodland remain and support a JJ sector in the southern portion of the state .
21 ['cls', 'many', 'n', '##ns', 'of', 'woodland', 'remain', 'and', 'support', 'a', 'jj', 'sector', 'in', 'the', 'southern', 'portion', 'of', 'the', 'state', '.', 'sep']
torch.Size([1, 21, 30522])
(6, 12, 21, 21)
torch.Size([1, 21, 768])
tensor(-3811819.5000, grad_fn=<SumBackward0>)
torch.Size([1, 21, 768])


In [67]:
all_examples_x = []
all_examples_vp = []
all_examples_y = []

all_examples_attentions = []
all_examples_blankout_relevance = []
all_examples_grads = []
all_examples_inputgrads = []
n_batches = 1000

all_examples_accuracies = []

infl_eng = inflect.engine()
verb_infl, noun_infl = gen_inflect_from_vocab(infl_eng, '../InDist/notebooks/wiki.vocab')

test_data = task.databuilder.as_dataset(split='validation', batch_size=1)
for examples in tqdm(test_data):
    sentence = task.sentence_encoder().decode(examples['sentence'][0])
    
    verb_position = examples['verb_position'][0].numpy()+1  #+1 because of adding cls.
    verb_position = offset_convertor(examples['sentence'][0], verb_position, task.sentence_encoder(), tokenizer)
    
    sentence = ['cls']+tokenizer.tokenize(sentence)+['sep']
    
    all_examples_vp.append(verb_position)
    sentence[verb_position] = tokenizer.mask_token
    tf_input_ids = tokenizer.encode(sentence)
    input_ids = torch.tensor([tf_input_ids])
    

    
    s_shape = input_ids.shape
    batch_size, length = s_shape[0], s_shape[1]
    actual_verb = examples['verb'][0].numpy().decode("utf-8")
    inflected_verb = verb_infl[actual_verb] 


    actual_verb_index = tokenizer.encode(tokenizer.tokenize(actual_verb))[1]
    inflected_verb_index = tokenizer.encode(tokenizer.tokenize(inflected_verb))[1]

    all_examples_x.append(input_ids)
    embeded_inputs = torch.autograd.Variable(model.distilbert.embeddings(input_ids), requires_grad=True)
    predictions = model(inputs_embeds=embeded_inputs)
    logits = predictions[0][0]

    
        
    probs = torch.nn.Softmax(dim=-1)(logits)
    actual_verb_score = probs[verb_position][actual_verb_index]
    inflected_verb_score = probs[verb_position][inflected_verb_index]
    
    main_diff_score = actual_verb_score - inflected_verb_score
    
    all_examples_accuracies.append(main_diff_score > 0)
    
    main_diff_score.backward()
    grads = embeded_inputs.grad
    grad_scores = abs(np.sum(grads.detach().numpy(), axis=-1))
    input_grad_scores = abs(np.sum((grads * embeded_inputs).detach().numpy(), axis=-1))
    all_examples_grads.append(grad_scores)
    all_examples_inputgrads.append(input_grad_scores)
    
    hidden_states, attentions = predictions[-2:]
    _attentions = [att.detach().numpy() for att in attentions]
    attentions_mat = np.asarray(_attentions)[:,0]

    all_examples_attentions.append(attentions_mat)
    
    # Repeating examples and replacing one token at a time with unk
    batch_size = 1
    max_len = input_ids.shape[1]
    
    # Repeat each example 'max_len' times
    x = input_ids
    extended_x = np.reshape(np.tile(x[:,None,...], (1, max_len, 1)),(-1,x.shape[-1]))

    # Create unk sequences and unk mask
    unktoken = tokenizer.encode([tokenizer.mask_token])[1]
    unks = unktoken * np.eye(max_len)
    unks =  np.tile(unks, (batch_size, 1))
    
    unk_mask =  (unktoken - unks)/unktoken
  
    # Replace one token in each repeatition with unk
    extended_x = extended_x * unk_mask + unks
    
    # Get the new output
    extended_predictions = model(torch.tensor(extended_x, dtype=torch.int64))
    extended_logits = extended_predictions[0]
    extended_probs = torch.nn.Softmax(dim=-1)(extended_logits)
    
    extended_correct_probs = extended_probs[:,verb_position,actual_verb_index]
    extended_wrong_probs =  extended_probs[:,verb_position,inflected_verb_index]
    extended_diff_scores = extended_correct_probs - extended_wrong_probs
    
    # Save the difference in the probability predicted for the correct class
    diffs = abs(main_diff_score - extended_diff_scores)

    all_examples_blankout_relevance.append(diffs.detach())
    n_batches -= 1
    if n_batches <= 0:
        break




INFO:absl:Constructing tf.data.Dataset for split validation, from ../InDist/data/word_sv_agreement/0.1.0



0it [00:00, ?it/s]


1it [00:00,  2.85it/s]


2it [00:00,  2.57it/s]


3it [00:01,  2.73it/s]


4it [00:01,  2.87it/s]


5it [00:01,  3.04it/s]


6it [00:02,  2.33it/s]


7it [00:02,  2.38it/s]


8it [00:03,  2.59it/s]


9it [00:03,  2.79it/s]


10it [00:03,  3.02it/s]


11it [00:04,  2.67it/s]


12it [00:04,  2.80it/s]


13it [00:04,  2.74it/s]


14it [00:05,  2.93it/s]


15it [00:05,  2.65it/s]


16it [00:06,  2.52it/s]


17it [00:06,  2.63it/s]


18it [00:06,  2.50it/s]


19it [00:07,  2.72it/s]


20it [00:07,  3.11it/s]


21it [00:07,  3.27it/s]


22it [00:07,  3.21it/s]


23it [00:08,  3.27it/s]


24it [00:08,  2.65it/s]


25it [00:09,  2.14it/s]


26it [00:09,  2.40it/s]


27it [00:10,  2.43it/s]


28it [00:10,  2.51it/s]


29it [00:10,  2.44it/s]


30it [00:11,  2.63it/s]


31it [00:11,  2.22it/s]


32it [00:12,  2.48it/s]


33it [00:12,  2.62it/s]


34it [00:13,  2.29it/s

In [68]:
def get_raw_att_relevance(full_att_mat, input_tokens, layer=-1, output_index=0):
    raw_rel = full_att_mat[layer].sum(axis=0)[output_index]/full_att_mat[layer].sum(axis=0)[output_index].sum()
    
    return raw_rel


def get_joint_relevance(full_att_mat, input_tokens, layer=-1, output_index=0):
    att_sum_heads =  full_att_mat.sum(axis=1) / full_att_mat.shape[1]
    joint_attentions = compute_joint_attention(att_sum_heads, add_residual=True)
    relevance_attentions = joint_attentions[layer][output_index]
    return relevance_attentions


def get_flow_relevance(full_att_mat, input_tokens, layer, output_index):
    
    input_tokens = input_tokens
    res_att_mat = full_att_mat.sum(axis=1)/full_att_mat.shape[1]
    res_att_mat = res_att_mat + np.eye(res_att_mat.shape[1])[None,...]
    res_att_mat = res_att_mat / res_att_mat.sum(axis=-1)[...,None]

    res_adj_mat, res_labels_to_index = get_adjmat(mat=res_att_mat, input_tokens=input_tokens)
    
    A = res_adj_mat
    res_G=nx.from_numpy_matrix(A, create_using=nx.DiGraph())
    for i in np.arange(A.shape[0]):
        for j in np.arange(A.shape[1]):
            nx.set_edge_attributes(res_G, {(i,j): A[i,j]}, 'capacity')


    output_nodes = ['L'+str(layer+1)+'_'+str(output_index)]
    input_nodes = []
    for key in res_labels_to_index:
        if res_labels_to_index[key] < full_att_mat.shape[-1]:
            input_nodes.append(key)
    
    flow_values = compute_node_flow(res_G, res_labels_to_index, input_nodes=input_nodes, output_nodes=output_nodes, length=full_att_mat.shape[-1])
    
    n_layers = full_att_mat.shape[0]
    length = full_att_mat.shape[-1]
    final_layer_attention = flow_values[(layer+1)*length:, layer*length:(layer+1)*length]
    relevance_attention_flow = final_layer_attention[output_index]

    return relevance_attention_flow

In [69]:
all_examples_attentions[0].shape

(6, 12, 24, 24)

In [ ]:
print("compute raw relevance scores ...")
all_examples_raw_relevance = {}
for l in np.arange(0,6):
    all_examples_raw_relevance[l] = []
    for i in tqdm(np.arange(len(all_examples_x))):
        tokens = tokenizer.decode(all_examples_x[i][0].numpy())
        vp = all_examples_vp[i]
        length = len(tokens)
        attention_relevance = get_raw_att_relevance(all_examples_attentions[i], tokens, layer=l, output_index=vp)
        all_examples_raw_relevance[l].append(np.asarray(attention_relevance))

print("compute joint relevance scores ...")
all_examples_joint_relevance = {}
for l in np.arange(0,6):
    all_examples_joint_relevance[l] = []
    for i in tqdm(np.arange(len(all_examples_x))):
        tokens = tokenizer.decode(all_examples_x[i][0].numpy())
        vp = all_examples_vp[i]
        length = len(tokens)
        attention_relevance = get_joint_relevance(all_examples_attentions[i], tokens, layer=l, output_index=vp)
        all_examples_joint_relevance[l].append(np.asarray(attention_relevance))
    
print("compute flow relevance scores ...")
all_examples_flow_relevance = {}
for l in np.arange(0,6):
    all_examples_flow_relevance[l] = []
    for i in tqdm(np.arange(len(all_examples_x))):
        tokens = tokenizer.decode(all_examples_x[i][0].numpy())
        vp = all_examples_vp[i]
        length = len(tokens)
        attention_relevance = get_flow_relevance(all_examples_attentions[i], tokens, layer=l, output_index=vp)
        all_examples_flow_relevance[l].append(np.asarray(attention_relevance))





100%|██████████| 1000/1000 [00:00<00:00, 14859.03it/s]




100%|██████████| 1000/1000 [00:00<00:00, 15522.56it/s]




100%|██████████| 1000/1000 [00:00<00:00, 15729.21it/s]






compute raw relevance scores ...


100%|██████████| 1000/1000 [00:00<00:00, 15680.46it/s]




100%|██████████| 1000/1000 [00:00<00:00, 15634.52it/s]




100%|██████████| 1000/1000 [00:00<00:00, 15488.00it/s]




  0%|          | 0/1000 [00:00<?, ?it/s]



 47%|████▋     | 471/1000 [00:00<00:00, 4702.15it/s]

compute joint relevance scores ...






100%|██████████| 1000/1000 [00:00<00:00, 4678.97it/s][A




  0%|          | 0/1000 [00:00<?, ?it/s]



 49%|████▊     | 487/1000 [00:00<00:00, 4863.21it/s]



100%|██████████| 1000/1000 [00:00<00:00, 4861.32it/s][A




  0%|          | 0/1000 [00:00<?, ?it/s]



 49%|████▉     | 494/1000 [00:00<00:00, 4927.54it/s]



100%|██████████| 1000/1000 [00:00<00:00, 4843.79it/s][A




  0%|          | 0/1000 [00:00<?, ?it/s]



 47%|████▋     | 466/1000 [00:00<00:00, 4654.21it/s]



100%|██████████| 1000/1000 [00:00<00:00, 4673.44it/s][A




  0%|          | 0/1000 [00:00<?, ?it/s]



 47%|████▋     | 472/1000 [00:00<00:00, 4713.53it/s]



100%|██████████| 1000/1000 [00:00<00:00, 4673.90it/s][A




  0%|          | 0/1000 [00:00<?, ?it/s]



 48%|████▊     | 475/1000 [00:00<00:00, 4738.94it/s]



100%|██████████| 1000/1000 [00:00<00:00, 4767.73it/s][A




  0%|          | 0/1000 [00:00<?, ?it/s]

compute flow relevance scores ...






  0%|          | 1/1000 [00:00<09:40,  1.72it/s]



  0%|          | 2/1000 [00:05<30:54,  1.86s/it]



  0%|          | 3/1000 [00:05<22:06,  1.33s/it]



  0%|          | 4/1000 [00:05<17:34,  1.06s/it]



  0%|          | 5/1000 [00:06<14:11,  1.17it/s]



  1%|          | 6/1000 [00:14<50:10,  3.03s/it]



  1%|          | 7/1000 [00:15<42:48,  2.59s/it]



  1%|          | 8/1000 [00:16<32:59,  2.00s/it]



  1%|          | 10/1000 [00:17<25:01,  1.52s/it]



  1%|          | 11/1000 [00:20<32:02,  1.94s/it]



  1%|          | 12/1000 [00:20<23:52,  1.45s/it]



  1%|▏         | 13/1000 [00:20<17:50,  1.08s/it]



  1%|▏         | 14/1000 [00:21<17:24,  1.06s/it]



  2%|▏         | 15/1000 [00:24<27:16,  1.66s/it]



  2%|▏         | 16/1000 [00:27<30:42,  1.87s/it]



  2%|▏         | 17/1000 [00:27<22:04,  1.35s/it]



  2%|▏         | 18/1000 [00:31<33:02,  2.02s/it]



  2%|▏         | 19/1000 [00:31<24:13,  1.48s/it]



  2%|▏         | 21/1000 [00:31<18:25,  1.13s/it]


In [ ]:
raw_sps_blank = []
raw_sps_grad = []
raw_sps_inputgrad = []

joint_sps_blank = []
joint_sps_grad = []
joint_sps_inputgrad = []

flow_sps_blank = []
flow_sps_grad = []
flow_sps_inputgrad = []


for l in np.arange(0,6):
    print("###############Layer ",l, "#############")
    print('raw blankout')
    for i in np.arange(len(all_examples_x)):
        sp = spearmanr(all_examples_raw_relevance[l][i],all_examples_blankout_relevance[i].numpy())
        if not math.isnan(sp[0]):
            raw_sps_blank.append(sp[0])
        else:
            raw_sps_blank.append(0)
        
    print(np.mean(raw_sps_blank))
    
    
    print('raw inputgrad')
    print(all_examples_raw_relevance[l][0].shape, all_examples_inputgrads[0][0].shape)
    for i in np.arange(len(all_examples_x)):
        sp = spearmanr(all_examples_raw_relevance[l][i],all_examples_inputgrads[i][0])
        if not math.isnan(sp[0]):
            raw_sps_inputgrad.append(sp[0])
        else:
            raw_sps_inputgrad.append(0)
        
    print(np.mean(raw_sps_inputgrad))
    
    print('raw grad')
    print(all_examples_raw_relevance[l][0].shape, all_examples_grads[0][0].shape)
    for i in np.arange(len(all_examples_x)):
        sp = spearmanr(all_examples_raw_relevance[l][i],all_examples_grads[i][0])
        if not math.isnan(sp[0]):
            raw_sps_grad.append(sp[0])
        else:
            raw_sps_grad.append(0)
        
    print(np.mean(raw_sps_grad))
    
    print('joint blankout')
    for i in np.arange(len(all_examples_x)):
        sp = spearmanr(all_examples_joint_relevance[l][i],all_examples_blankout_relevance[i].numpy())
        if not math.isnan(sp[0]):
            joint_sps_blank.append(sp[0])
        else:
            joint_sps_blank.append(0)
        
    print(np.mean(joint_sps_blank))
    
    print('joint grad')
    print(all_examples_joint_relevance[l][0].shape, all_examples_grads[0][0].shape)
    for i in np.arange(len(all_examples_x)):
        sp = spearmanr(all_examples_joint_relevance[l][i],all_examples_grads[i][0])
        if not math.isnan(sp[0]):
            joint_sps_grad.append(sp[0])
        else:
            joint_sps_grad.append(0)
        
    print(np.mean(joint_sps_grad))
    
    print('joint inputgrad')
    print(all_examples_joint_relevance[l][0].shape, all_examples_inputgrads[0][0].shape)
    for i in np.arange(len(all_examples_x)):
        sp = spearmanr(all_examples_joint_relevance[l][i],all_examples_inputgrads[i][0])
        if not math.isnan(sp[0]):
            joint_sps_inputgrad.append(sp[0])
        else:
            joint_sps_inputgrad.append(0)
        
    print(np.mean(joint_sps_inputgrad))
    
    print('flow')
    for i in np.arange(len(all_examples_x)):
        sp = spearmanr(all_examples_flow_relevance[l][i],all_examples_blankout_relevance[i].numpy())
        
        if not math.isnan(sp[0]):
            flow_sps_blank.append(sp[0])
        else:
            flow_sps_blank.append(0)
        
    print(np.mean(flow_sps_blank))
  
    print('flow grad')
    print(all_examples_joint_relevance[l][0].shape, all_examples_grads[0][0].shape)
    for i in np.arange(len(all_examples_x)):
        sp = spearmanr(all_examples_flow_relevance[l][i],all_examples_grads[i][0])
        if not math.isnan(sp[0]):
            flow_sps_grad.append(sp[0])
        else:
            flow_sps_grad.append(0)
        
    print(np.mean(flow_sps_grad))
    
    print('flow inputgrad')
    print(all_examples_joint_relevance[l][0].shape, all_examples_inputgrads[0][0].shape)
    for i in np.arange(len(all_examples_x)):
        sp = spearmanr(all_examples_flow_relevance[l][i],all_examples_inputgrads[i][0])
        if not math.isnan(sp[0]):
            flow_sps_inputgrad.append(sp[0])
        else:
            flow_sps_inputgrad.append(0)
        
    print(np.mean(flow_sps_inputgrad))
        

# for l in np.arange(6):
#     print("layer ",l)
#     print(all_examples_blankout_relevance[0].numpy().shape, all_examples_raw_relevance[l][0].shape, all_examples_joint_relevance[l][0].shape)
#     print('raw:',np.mean([spearmanr(all_examples_blankout_relevance[i], all_examples_blankout_relevance[i].numpy()) for i in np.arange(len(all_examples_x))]))
#     print('joint',np.mean([spearmanr(all_examples_joint_relevance[l][i], all_examples_blankout_relevance[i].numpy()) for i in np.arange(len(all_examples_x))]))
#     #print('flow',np.mean([spearmanr(all_examples_flow_relevance[l][i], all_examples_blankout_relevance[l][i]) for i in np.arange(len(all_examples_x))]))

In [60]:
print(all_examples_joint_relevance[l][0].shape)
print(all_examples_flow_relevance[l][0].shape)
print(all_examples_blankout_relevance[0].numpy().shape)
print(all_examples_inputgrads[0][0].shape)

(24,)
(24,)
(24,)
(24,)


In [61]:
all_examples_inputgrads[0][0]

array([8.1960952e-06, 1.4568408e-05, 5.1740431e-06, 1.4364506e-06,
       4.1196945e-06, 1.5648286e-06, 4.4366584e-06, 1.2030331e-06,
       6.7950618e-06, 3.5287808e-06, 2.4310439e-06, 7.5509070e-06,
       1.3048927e-06, 4.7139088e-06, 5.4677326e-07, 3.7762169e-05,
       8.8991792e-06, 3.1733543e-06, 4.0503464e-06, 7.9696765e-06,
       4.9376549e-06, 4.7843009e-06, 2.7463548e-06, 1.0299571e-05],
      dtype=float32)